In [36]:
import pandas as pd

dm1 = pd.read_csv('../data/external/demographics/dem_fullpilot12.csv')
dm2 = pd.read_csv('../data/external/demographics/dem_fullpilot13.csv')
dm3 = pd.read_csv('../data/external/demographics/dem_fullpilot14.csv')



df1 = pd.read_csv('../data/raw/fullpilot13_2.csv')
df2 = pd.read_csv('../data/raw/fullpilot1466.csv')


df2 = df2[df2.expName.isin(['FullPilot14'])]
df1 = df1[df1.expName.isin(['FullPilot13', 'FullPilot12'])]

df1 = df1.groupby('prolificID').filter(lambda x: len(x) >= 483)
df2 = df2.groupby('prolificID').filter(lambda x: len(x) >= 483)
# df = df.groupby('prolificID').filter(lambda x: len(x) <= 432)
# keep  prolificIDs that are more than 10 characters
df1 = df1[df1.prolificID.str.len() > 10]
df2 = df2[df2.prolificID.str.len() > 10]
# show prolificIDs 

In [38]:
dm1['prolificID'] = dm1['Participant id']
dm2['prolificID'] = dm2['Participant id']
dm3['prolificID'] = dm3['Participant id']

In [45]:
pids1 = df1.prolificID.unique()
pids2 = df2.prolificID.unique()

# check that pids1 are in dm1 and pids2 are in dm2
dm1 = dm1[(dm1.prolificID.isin(pids1)) | (dm1.prolificID.isin(pids2))]
dm2 = dm2[(dm2.prolificID.isin(pids1)) | (dm2.prolificID.isin(pids2))]
dm3 = dm3[(dm3.prolificID.isin(pids1)) | (dm3.prolificID.isin(pids2))]

In [40]:
print(dm1.shape)
print(dm2.shape)

(62, 21)
(64, 21)


In [46]:
# check dm1 and dm2 do not have the same prolificIDs
dm1 = dm1[~dm1.prolificID.isin(dm2.prolificID)]
dm2 = dm2[~dm2.prolificID.isin(dm1.prolificID)]
dm3 = dm3[~dm3.prolificID.isin(dm1.prolificID)]
dm3 = dm3[~dm3.prolificID.isin(dm2.prolificID)]


In [47]:
print(dm1.shape)
dm2.shape

(61, 21)


(64, 21)

In [48]:
dm3.shape

(65, 21)